# Preparing protein structures for simulation: Reduce and ProPka.

In this tutorial you will see how you can use [Reduce](http://kinemage.biochem.duke.edu/software/reduce.php) and [Propka](https://github.com/jensengroup/propka-3.1) in the preparation and validation of a protein structure for molecular simulation.

* `Reduce` analyses protein PDB format files and predicts the coordinates of missing hydrogen atoms. It can also correct for "NQH flips". However `reduce` assumes all amino acids are in their default ionization states.
* `Propka` analyses protein PDB format files and predicts the pKas of titratable groups. From this information you can predict if a particular sidechain is likely to be ionized or not at the pH you have chosen for the simulation. However `propka` does not actually add any H-atoms to your molecular structure.

You will see what each of these two tools detects when it analyses the crystal structure of the cysteine protease cruzein (pdb code [2OZ2](https://www.rcsb.org/structure/2OZ2))

Finally you will see how the [PDB2PQR webserver](http://server.poissonboltzmann.org/pdb2pqr) provides a very useful way to apply both these tools to a protein structure.

 
## Orientation:

This Jupyter notebook window contains the tutorial guide, and is where you will visualise the protein structure. To the right is a unix terminal window; this is where you will execute the commands to run the `reduce` and `propka` tools, and look at the output they produce. Be aware that this unix window is connected to the CCP-BioSim cloud service - you are not running on your own laptop/desktop.

## Part 1: visualisation of the protein.

In this cell we load the pdb file and visualize it using nglviewer:

In [ ]:
import nglview as nv
import mdtraj as mdt
import time
pdb_file = 'data/2oz2_A.pdb'
view = nv.show_mdtraj(mdt.load(pdb_file))
view.add_representation('ball+stick', 'acidic')
view.add_representation('ball+stick', 'basic')
view

The view shows most of the protein in ribbon form, but titratable sidechains arwe shown in full - as these are likely to be the most "interesting". Note that at the moment the structure contains no hydrogen atoms.

## Part 2: Hydrogen atom prediction using Reduce

In the terminal window to the right, use `ls`,`cd` etc. to make sure you are in the `data` directory. You should be able to see the file `2oz2_A.pdb`.

type:

    reduce -h
    
You should see an extensive documentation of the program. We will use the '-FLIP' option so that as well as predicting where H-atoms are, potential flips in the orientations of N, Q, and H sidechains are analysed.

run reduce:

    reduce -FLIP 2oz2_A.pdb > 2oz2_A_fh.pdb
    
A lot of info is written to the screen, and a new pdb format file `2oz2_A_fh.pdb` is produced. The info written to the screen is probably only of interest to a specialist or software developer, the new pdb file contains everything you are interested in for now. In the terminal window, browse through this file using `nano` or maybe just `more`.

The top section contains USER lines that `reduce` has added, summarising what it has found and done. After that comes the protein structure. Newly-added hydrogen atoms are marked by having "new" at the end of the line, and an index number (second column) of zero, e.g. here is part of the file:

```
USER  MOD Single : A 198 LYS NZ  :NH3+   -171:sc=    1.06   (180deg=0.949)
USER  MOD Single : A 201 ASN     :      amide:sc=  0.0747  K(o=0.075,f=-5.1!)
USER  MOD Single : A 206 LYS NZ  :NH3+    180:sc=       0   (180deg=0)
USER  MOD -----------------------------------------------------------------
ATOM      1  N   ALA A   1      33.649  -5.901  -7.706  1.00 18.36           N
ATOM      2  CA  ALA A   1      32.983  -4.882  -8.555  1.00 17.26           C
ATOM      3  C   ALA A   1      32.955  -3.550  -7.808  1.00 16.80           C
ATOM      4  O   ALA A   1      33.101  -3.533  -6.586  1.00 16.79           O
ATOM      5  CB  ALA A   1      31.542  -5.351  -8.932  1.00 17.52           C
ATOM      0  H1  ALA A   1      34.406  -6.163  -8.095  1.00 18.36           H   new
ATOM      0  H2  ALA A   1      33.831  -5.550  -6.909  1.00 18.36           H   new
```


Concentrate on what `reduce` has concluded about the histidine (HIS) residues in the protein. For each it has decided whether it is more likely to be the delta-tautomer (with an H atom on ND1, called "HD1"), or the epsilon-tautomer (with an H atom on NE2, called "HE2").

What decisions has it made? You can either look through the pdb file in the text editor in the terminal window, or you can use the visualisation below (hover the cursor over atoms to see their ID). 

In [ ]:
pdb_file2 = 'data/2oz2_A_fh.pdb'
view2 = nv.show_mdtraj(mdt.load(pdb_file2))
view2.add_representation('ball+stick', 'HIS')
view2

### Further study:

`Reduce` makes decisions about HIS tautomers based on which would produce the optimal H-bonding interactions with surrounding residues. Explore the environment around each HIS residue and see if you can convince yourself that `reduce` has indeed done this.

---------

## Part 3: pKa prediction using Propka

`Reduce` assumes all amino acids are in their default ionization state, so for a HIS residue it will test whether the delta- or epsilon-tautomer is more likely (based on H-bonding interactions), but it will not consider the possibility that the HIS residue is actually in the protonated form (with H atoms on both ND1 and NE2). For this we need to use `propka`.


In the terminal window type:

    propka3 -h
    
to see this list of options available to you. For now we will keep things very simple. Type:

    propka3 2oz2_A.pdb
    
As with reduce, a certain amount of information is written to the screen, and a new file -`2oz2_A.pka` - produced. But this case we are more interested (for now) in what was written to the screen. The last section "SUMMARY OF THIS PREDICTION" contains the key data.

This list the predicted pKa of each titratable group in the protein. Concentrate for now on the aspartic acid (ASP) and glutamic acid (GLU) residues. Carboxylic acids are expected to have pKas in the 3.5-4.5 range, i.e., to lose their carboxylic acid proton if the pH is greater than 3.5-4.5, and so be ionised at physiological pH (c. 7.4). Are all the ASP and GLU residues predicted to be ionized? If not, can you work out why?

If you run the code in the next cell, it will change the view of the protein in the cell above to zoom in on this region to help you answer this question.

In [ ]:
pdb_file3 = 'data/2oz2_A.pdb'
view3 = nv.show_mdtraj(mdt.load(pdb_file3))
view3.add_representation('ball+stick', 'acidic')
view3.add_representation('label', '57:A.CB')
view3.add_representation('label', '60:A.CB')
view3.add_representation('label', '50:A.CB')
view3.center('50:A')
view3

## Further study
What about the HIS residues: are they predicted to be in the protonated or neutral form?

---------

# Part 4: Using the PDB2PQR webserver

You will have seen that both `reduce` and `propka` tell you important things about this protein that must be taken into consideration when preparing it for simulation. But there are problems:

* `reduce` tells you about things like delta versus epsilon tautomers of HIS residues, and produces a new PDB format file with hydrogens added, but does not analyse pKa values.

* `propka` tells you about likely protonation states of titratable residues, but does not produce a new PDB format file with this extra data.


The preparation programs that come with most MD packages have the ability to correctly build  protein models if they are given a PDB format file to start with that contains the right information. This is typically encoded through modifications to the three letter residue codes for the variable amino acids. So for example, to tell a modelling program that a particular histidine residue is supposed to be the delta tautomer, you would modify the name of that residue in the PDB file from "HIS" to "HID". An epsilon tautomer would be "HIE" and a protonated histidine "HIP". There are modified names for, e.g. neutral forms of aspartic and glutamic acid as well ("ASH" and "GLH"). 

Between them, `reduce` and `propka` give you all the information necessary to do this, but don't actually complete the job themselves. You could edit the PDB file by hand, but the [PDB2PQR webserver](http://server.poissonboltzmann.org/pdb2pqr) can do this for you.

If you have not already done so, open a new tab in your browser to the PDB2PQR webserver.

1. Near the top you will see there are two options: to have the program analyse a PDB file already in the Protein Data Bank by just inputting its four-character PDB ID, or to have it analyse a PDB format file that you upload yourself. We will go for the first option, so enter the PDB ID "2oz2" into the relevant box.

2. Leave the pKa options section at the default values, so PROPKA will be used to calculate pKas.

3. The server will generate a new PDB format file for your protien, with the "right" added hydrogens and "right" amino acid names. However different MD codes expect slightly different data here, so you need to select what's needed by your prefered package. For now, assume that is "AMBER" so select that in both sections below.

4. Leave all other options as they are, just click "Start Job".

5. When the job finishes (it should just take 30 seconds or so, though maybe longer if lots of people are trying to use it at the same time) you will see a list of output files available for download, one of which ends in ".pqr". This is actually a PDB format file, and the one you want.

6. **Do not download this file!** - it would be downloaded to the laptop/workstation you are actually sitting on, and remember your terminal window is actually connected to the CCP-BioSim cloud service. So instead look in the `pdb2pqr` folder, and you will see a file `2oz2_pqr.pdb` there, which is a copy of the same thing.

7. Browse through this file (use `nano` or `more`, as you like). At the top in the REMARK section is info about what PDb2PQR did. Then below this is the atom and residue data. Compare what's in this file with the conclusions you came to from doing parts 2 and 3 of this tutorial - does it all add up?

# Summary
Take care! Adding the hydrogen atoms to a protein structure that lacks them is not a trivial exercise. If you make an error here it can mean everything you do after is invalid.

Fortunately it's easy to access good tools that can help you avoid the pitfalls.


*Charlie Laughton, September 2020*

